Script to do meta-connectivity in two parts. First part: Homotopic connectivity analysis. Second part: cluster analysis

In [ ]:
#!/usr/bin/python
from turtle import color
import pandas as pd
import numpy as np
import sys
sys.path.append('/mnt/c/Users/Wayne/tvb/Network-science-Toolbox/Python')
# sys.path.append('/home/wayne/github/TVB_workflow/new_g_optimal')
# sys.path.append('/home/wayne/github/Network-science-Toolbox/Python')
from TS2dFCstream import TS2dFCstream
from dFCstream2Trimers import dFCstream2Trimers
from dFCstream2MC import dFCstream2MC
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.stats import pearsonr
import itertools
import os
import scipy.io

"""
@Author: Yile Wang

This script is used to calculate the homotopic meta-connectivity in four groups, SNC, NC, MCI, AD
"""

# brain region labels for your reference
regions = ['aCNG-L', 'aCNG-R','mCNG-L','mCNG-R','pCNG-L','pCNG-R', 'HIP-L','HIP-R','PHG-L','PHG-R','AMY-L','AMY-R', 'sTEMp-L','sTEMP-R','mTEMp-L','mTEMp-R']
regionswithgroups = ['groups','aCNG-L', 'aCNG-R','mCNG-L','mCNG-R','pCNG-L','pCNG-R', 'HIP-L','HIP-R','PHG-L','PHG-R','AMY-L','AMY-R', 'sTEMp-L','sTEMP-R','mTEMp-L','mTEMp-R']
group = ['SNC', 'NC', 'MCI','AD']
regionsHalf = np.array(['aCNG', 'mCNG','pCNG','HIP','PHG','AMY','sTEMp', 'mTEMp'])

regions14 = []
for i in range(14):
    wt = ["regions_", str(i)]
    wt = "".join(wt)
    regions14.append(wt)

# iterate simulated functional connectivity
if __name__ == "__main__":
    Trimer_Homo = pd.DataFrame(columns=['groups','trimer_homo','aCNG','mCNG','pCNG','HIP','PHG','AMY','sTEMp','mTEMp' ])
    Trimer_Hetero  = pd.DataFrame(columns=['groups','trimer_hetero','aCNG','mCNG','pCNG','HIP','PHG','AMY','sTEMp','mTEMp' ])
    Trimer = pd.DataFrame()
    
    for grp in group:
        # subject case ids
        ldir = os.listdir("/mnt/c/Users/Wayne/tvb/TS-4-Vik/"+ grp+'-TS')
        # ldir = os.listdir('/home/wayne/TS-4-Vik/'+grp+'-TS/')
        MC_all = np.zeros((16,16, 16, len(ldir)))
        tmp_homo = np.array([])
        homoRegions = np.ones((1,len(regionsHalf)))
        for ind, y in enumerate(ldir):
            # import empirical functional connectivity
            # Here is the path of the mat file of the FC data
            pth_efc = "/mnt/c/Users/Wayne/tvb/TS-4-Vik/"+ grp+'-TS/'+ y +"/ROISignals_"+ y +".mat"
            # pth_efc = "/home/wayne/TS-4-Vik/"+grp+"-TS/"+ y +"/ROISignals_"+ y +".mat"
            ea = scipy.io.loadmat(pth_efc)
            all = ea['ROISignals']
            df = pd.DataFrame.from_dict(all)
            df.columns = regions
            # calculate the meta-connectivity, using existing script:
            dFCstream = TS2dFCstream(df.to_numpy(), 5, None, '2D')
            # Calculate MC
            MC_MC = dFCstream2MC(dFCstream)
            # Calculate Trimers results, with nxnxn information
            MC_Trimers = dFCstream2Trimers(dFCstream)
            MC_all[:,:,:,ind] = MC_Trimers
        MC_homo = np.mean(MC_all, 3)
        MC_single_groups = np.zeros((14, 8))
        # only pick up L sides of the regions
        for idnx, i in enumerate(range(0,15,2)):
            j = i+1 # represent R side
            newList = list(range(16))
            del newList[i:j+1] # drop the target regions L and R
            for idx, restNode in enumerate(newList):
                MC_single_groups[idx,idnx] = MC_homo[i,j,restNode] # In rest of the 14 regions, iternate the third dimensions, and pick up the homotopic MC
        MC_single = pd.DataFrame(MC_single_groups, index=regions14, columns=regionsHalf)
        grpInfo = [grp] * 14
        MC_single.insert(0, "group", grpInfo)
        Trimer = Trimer.append(MC_single)
    print(Trimer)
    # Trimer.to_excel("/mnt/c/Users/Wayne/tvb/gc1sec_res/mc_homo.xlsx")

In [ ]:
# calculate the cluster
